# Data processing
The goal of this notebook is to convert data to JSON format that can be ingested by force directed graph visualization on web

In [1]:
import pandas as pd

### Load Ingredient-Ingredient data

Example of output JSON format:
````
{
  "nodes": [
    {"id": "alginate", "group": 1},
    {"id": "sodium", "group": 1},
    {"id": "sodium alginate", "group": 1},
    {"id": "agar", "group": 1},
    {"id": "carrageenan", "group": 1}
  ],
  "links": [
    {"source": "alginate", "target": "sodium", "value": 1},
    {"source": "sodium alginate", "target": "agar", "value": 8},
    {"source": "carrageenan", "target": "agar", "value": 10}
  ]
}
````


In [11]:
ing_ing = pd.read_csv('ingredient-ingredient.csv')
ing_ing.head(10)

,first,second,times,docids
0,alginate,alginate,547,"[3, 5, 6, 7, 10, 12, 13, 14, 15, 16, 17, 18, 1..."
1,alginate,sodium,532,"[3, 6, 10, 12, 14, 16, 19, 20, 21, 23, 24, 25,..."
2,alginate,sodium alginate,508,"[3, 6, 10, 12, 14, 16, 19, 20, 21, 23, 24, 25,..."
3,sodium,sodium alginate,508,"[3, 6, 10, 12, 14, 16, 19, 20, 21, 23, 24, 25,..."
4,sodium,sodium,200,"[3, 6, 10, 11, 12, 14, 16, 19, 20, 21, 23, 24,..."
5,alginate,calcium,185,"[5, 7, 13, 29, 33, 62, 79, 97, 98, 116, 118, 1..."
6,carrageenan,carrageenan,181,"[8, 34, 39, 45, 46, 57, 58, 59, 60, 61, 70, 72..."
7,alginate,chitosan,179,"[5, 10, 17, 23, 30, 35, 52, 101, 127, 138, 146..."
8,agar,agar,167,"[0, 1, 2, 4, 9, 11, 27, 32, 36, 40, 49, 50, 51..."
9,sodium alginate,sodium alginate,148,"[3, 6, 10, 12, 14, 16, 19, 20, 21, 23, 24, 25,..."


### Load Ingredient-Property data

In [7]:
ing_prop = pd.read_excel('ingredient-property.xlsx') 
ing_prop.head()

,ingredient,property,times,docids
0,alginate,tensile strength,320,"[6, 13, 14, 16, 19, 20, 21, 28, 29, 30, 38, 42..."
1,alginate,water vapor,201,"[3, 6, 7, 13, 16, 17, 18, 19, 22, 42, 62, 67, ..."
2,sodium,tensile strength,196,"[6, 11, 14, 16, 19, 20, 21, 29, 38, 42, 67, 68..."
3,sodium alginate,tensile strength,182,"[6, 14, 16, 19, 20, 21, 29, 38, 42, 67, 68, 69..."
4,alginate,thickness,149,"[14, 23, 33, 85, 92, 100, 134, 140, 189, 195, ..."


### Load Abstracts Database

In [10]:
abstracts_database = pd.read_excel('article-abstract-doi-database.xlsx') 
abstracts_database.head()

,ID,Authors,Title,Year,DOI,Link,Abstract,DOWNLOAD
0,0,Marvizadeh M.M.; Tajik A.; Moosavian V.; Oladz...,Fabrication of cassava starch/mentha piperita ...,2021.0,10.22034/jchr.2020.1900584.1135,https://www.scopus.com/inward/record.uri?eid=2...,In this study; an edible film is prepared usin...,1.0
1,1,Yu G.; Zhang Q.; Wang Y.; Yang Q.; Yu H.; Li H...,Sulfated polysaccharides from red seaweed Geli...,2021.0,10.1016/j.foodhyd.2021.106820,https://www.scopus.com/inward/record.uri?eid=2...,Marine red algae are rich sources of sulfated ...,1.0
2,2,Wang B.; Yan S.; Gao W.; Kang X.; Yu B.; Liu P...,Antibacterial activity; optical; and functiona...,2021.0,10.1016/j.foodchem.2021.129743,https://www.scopus.com/inward/record.uri?eid=2...,In this study; the optical; morphological; ant...,1.0
3,3,Zhang J.; Huang X.; Shi J.; Liu L.; Zhang X.; ...,A visual bi-layer indicator based on roselle a...,2021.0,10.1016/j.foodchem.2021.129573,https://www.scopus.com/inward/record.uri?eid=2...,This study designed a new type indicator with ...,1.0
4,4,Roy S.; Rhim J.-W.,Fabrication of pectin/agar blended functional ...,2021.0,10.1016/j.foodhyd.2021.106823,https://www.scopus.com/inward/record.uri?eid=2...,Pectin/agar blended multifunctional composite ...,1.0
